In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2951, 거래소 1554, 코스닥 1397
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2951 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 326,547,075,400,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 63,991,407,883,500
삼성전자우 상장주식수: 822,886,700, 시가총액 36,947,612,830,000
NAVER 상장주식수: 164,813,395, 시가총액 28,924,750,822,500
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 26,201,340,000,000
현대차 상장주식수: 213,668,187, 시가총액 25,853,850,627,000
현대모비스 상장주식수: 95,306,694, 시가총액 24,827,393,787,000
셀트리온 상장주식수: 128,337,853, 시가총액 22,074,110

LIG넥스원 상장주식수: 22,000,000, 시가총액 737,000,000,000
후성 상장주식수: 92,606,819, 시가총액 735,298,142,860
SK케미칼 상장주식수: 11,741,396, 시가총액 731,488,970,800
동부제철 상장주식수: 100,008,897, 시가총액 728,064,770,160
삼천당제약 상장주식수: 22,105,813, 시가총액 727,281,247,700
롯데하이마트 상장주식수: 23,607,712, 시가총액 727,117,529,600
씨젠 상장주식수: 26,234,020, 시가총액 725,370,653,000
파트론 상장주식수: 54,156,410, 시가총액 722,988,073,500
애경산업 상장주식수: 26,409,935, 시가총액 710,427,251,500
동국제약 상장주식수: 8,892,000, 시가총액 705,135,600,000
아주캐피탈 상장주식수: 57,545,890, 시가총액 699,182,563,500
텔콘RF제약 상장주식수: 83,157,022, 시가총액 688,540,142,160
한세실업 상장주식수: 40,000,000, 시가총액 688,000,000,000
KINDEX 200 상장주식수: 23,450,000, 시가총액 685,326,250,000
비에이치 상장주식수: 32,680,468, 시가총액 681,387,757,800
ARIRANG 200 상장주식수: 23,200,000, 시가총액 678,368,000,000
보령제약 상장주식수: 44,200,000, 시가총액 676,260,000,000
티씨케이 상장주식수: 11,675,000, 시가총액 674,815,000,000
효성티앤씨 상장주식수: 4,327,682, 시가총액 670,790,710,000
제주항공 상장주식수: 26,356,758, 시가총액 670,779,491,100
동양생명 상장주식수: 161,358,585, 시가총액 662,376,991,425
영원무역홀딩스 상장주식수: 13,635,592, 시가총액 660,6

옵티시스 상장주식수: 5,636,000, 시가총액 44,411,680,000
백금T&A 상장주식수: 15,407,187, 시가총액 44,372,698,560
SK케미칼우 상장주식수: 1,458,670, 시가총액 44,343,568,000
아이엠 상장주식수: 42,223,170, 시가총액 44,334,328,500
TIGER 원유선물Enhanced(H) 상장주식수: 10,700,000, 시가총액 44,298,000,000
디엔에이링크 상장주식수: 11,476,541, 시가총액 44,242,065,555
캐스텍코리아 상장주식수: 16,569,188, 시가총액 44,239,731,960
KODEX 200가치저변동 상장주식수: 5,200,000, 시가총액 44,044,000,000
큐로홀딩스 상장주식수: 63,416,213, 시가총액 44,010,851,822
피에스케이홀딩스 상장주식수: 5,566,584, 시가총액 43,864,681,920
태양금속 상장주식수: 36,700,000, 시가총액 43,856,500,000
알티캐스트 상장주식수: 21,223,160, 시가총액 43,719,709,600
동국알앤에스 상장주식수: 18,400,000, 시가총액 43,700,000,000
나우IB 상장주식수: 9,480,000, 시가총액 43,655,400,000
네오티스 상장주식수: 11,013,678, 시가총액 43,614,164,880
한국종합기술 상장주식수: 10,950,000, 시가총액 43,526,250,000
테라셈 상장주식수: 15,678,688, 시가총액 43,508,359,200
위즈코프 상장주식수: 38,830,772, 시가총액 43,490,464,640
수산아이앤티 상장주식수: 6,751,000, 시가총액 43,408,930,000
이화산업 상장주식수: 2,800,000, 시가총액 43,400,000,000
지티지웰니스 상장주식수: 12,316,266, 시가총액 43,353,256,320
베셀 상장주식수: 11,192,368, 시가총액 43,258,502

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [15]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [16]:
import pandas as pd

In [17]:
format(100000000000, ",")

'100,000,000,000'

In [18]:
format(1000000000000, ",")

'1,000,000,000,000'

In [19]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [20]:
# pdMarketTotal = pdMarketTotal.loc[(
#     (((pdMarketTotal[0].apply(lambda x: x.startswith("KODEX"))) | (pdMarketTotal[0].apply(lambda x: x.startswith("TIGER")))) & (pdMarketTotal[3] > 200000000000)) |
#     (pdMarketTotal[3] > 2000000000000)
# )]

In [23]:
pdMarketTotal = pdMarketTotal[:200]

In [24]:
pdMarketTotal

,0,1,2,3
0,삼성전자,A005930,5969782,326547075400000
1,SK하이닉스,A000660,728002365,63991407883500
2,삼성전자우,A005935,822886700,36947612830000
3,NAVER,A035420,164813395,28924750822500
4,삼성바이오로직스,A207940,66165000,26201340000000
...,...,...,...,...
195,녹십자홀딩스,A005250,47028210,1036972030500
196,에코프로비엠,A247540,20555966,1026770501700
197,TRUE 코스피 양매도 5% OTM ETN,Q570019,98900000,1024604000000
198,코리안리,A003690,120369116,1015915339040


In [25]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization".split(",")

In [14]:
pdMarketTotal.to_csv("../data/KRX200_LIST.csv", index=None)